In [1]:
import torch
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10, Food101
from torchvision.transforms import v2
from pathlib import Path
import wandb

import sys

sys.path.append("../src")

from trainer import Trainer
from module import VGGNetModule
from utils import model_size
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger
from dataset import MapDataset

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "VGGNet",
        "batch_size": 64,
        "max_epochs": 100,
        "optimizer": {
            "name": "Adam",
        },
        "train_split": 42_000,
        "val_split": 8000
    },
    logs_path=logs_path
)

In [4]:
dataset = CIFAR10(data_path, train=True, download=True)

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]]
)

val_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_dataset = MapDataset(train_dataset, transform=v2.Compose([
    # Data auguments
    v2.RandomCrop(size=(32, 32), padding=4, padding_mode='reflect'),
    v2.RandomHorizontalFlip(),

    val_transforms
]))

val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True)
val_dataloader = DataLoader(
    val_dataset, batch_size=logger.config["batch_size"])

Files already downloaded and verified


In [5]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=90.0, accuracy_diff=5.0, wait_epochs=5),
    # OverfitCallback(limit_batches=2, max_epochs=500),
]

In [ ]:
module = VGGNetModule()
module.load_from_checkpoint("/root/ImageClassification/logs/wandb/run-20241208_005748-141pec1i/checkpoints/checkpoint-20.pt")

trainer = Trainer(
    module=module,
    logger=logger,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=False,
    measure_time=True
)

summary(module.model, input_size=train_dataset[0][0].shape, device="cpu")

model size: 21.66 MB


In [10]:
module.device

'cuda'

In [11]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Time per epoch: 21.66 seconds
Epoch: 0, train_accuracy: 90.95, val_accuracy: 91.68
Early stopping check started!
Epoch: 1, train_accuracy: 91.33, val_accuracy: 91.62
Epoch: 2, train_accuracy: 92.06, val_accuracy: 90.03
Epoch: 3, train_accuracy: 92.31, val_accuracy: 90.06
Epoch: 4, train_accuracy: 92.68, val_accuracy: 90.64
Epoch: 5, train_accuracy: 93.30, val_accuracy: 90.41
Epoch: 6, train_accuracy: 93.68, val_accuracy: 91.71
Epoch: 7, train_accuracy: 93.77, val_accuracy: 89.95
Epoch: 8, train_accuracy: 94.32, val_accuracy: 91.10
Epoch: 9, train_accuracy: 94.56, val_accuracy: 89.82
Epoch: 10, train_accuracy: 94.89, val_accuracy: 90.91
Epoch: 11, train_accuracy: 94.98, val_accuracy: 90.51
Epoch: 12, train_accuracy: 95.35, val_accuracy: 91.38
Epoch: 13, train_accuracy: 95.36, val_accuracy: 90.65
Epoch: 14, train_accuracy: 95.74, val_accuracy: 90.36
Epoch: 15, train_accuracy: 95.90, val_accuracy: 91.00
Epoch: 16, train_accuracy: 96.07, val_accuracy: 90.21
Epoch: 17, train_accuracy: 96.25

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
epoch_train_accuracy,▁▁▂▃▃▄▄▅▅▆▆▆▇▇▇▇▇███
epoch_train_loss,█▇▆▆▆▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁
epoch_val_accuracy,██▂▂▄▃█▂▆▁▅▄▇▄▃▅▃▄▂▁
epoch_val_loss,▁▁▅▄▄▅▂▆▃▆▄▅▄▅▆▅▆▇██
step,▆█▂▅▇▁▁▄▅▅▅▆▆▆▃▇▁▂▂▄▇█▅▄▇▁▂▄▅▃▄█▂▅▆▇▄▅▆▅
epoch,19
epoch_train_accuracy,96.51351
epoch_train_loss,0.10256
epoch_val_accuracy,89.7875
epoch_val_loss,0.34026


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)